In [86]:
import os
import glob
import cv2
from PIL import Image
from datetime import datetime
import time
from PIL.ExifTags import TAGS, GPSTAGS
import exifread


filepath = ('C:\\Users\\sami\\Desktop\\test images\\*.*') # change this path
list_images = []
NO =[]
images_location =[]
geotagging = {}
leak =[]

# Extract Images Location
def get_geotagging(exif):
    if not exif:
        raise ValueError("No EXIF metadata found")
    
    for (idx, tag) in TAGS.items():
        if tag == 'GPSInfo':
            if idx not in exif:
                raise ValueError("No EXIF geotagging found")

            for (key, val) in GPSTAGS.items():
                if key in exif[idx]:
                    geotagging[val] = exif[idx][key]                 

    return geotagging

def get_dms(dms):    
    degrees = dms[0][0] / dms[0][1]
    minutes = dms[1][0] / dms[1][1] 
    seconds = dms[2][0] / dms[2][1] 
    
    return round (degrees,2) , round (minutes,2) , round (seconds,2)

def get_coordinates_dms(geatags):
    lat = get_dms(geotags['GPSLatitude'])
    lon = get_dms(geotags['GPSLongitude'])

    return lat, geotags['GPSLatitudeRef'], lon, geotags['GPSLongitudeRef']


for bb,f1 in enumerate (glob.glob(filepath)):   
    img = cv2.imread(f1) # read image
    
    #Extract images names
    img_name = os.path.basename(f1)   
    list_images.append(img_name)
    
    #image loctaion
    exif = Image.open(f1)._getexif()
    geotags = get_geotagging(exif)
    images_location.append(get_coordinates_dms(geotags))
    
    
    if (bb < 1):
        im = Image.open(f1)
        exif = im.getexif()
        creation_time = exif.get(36867)
        onlyDate = ('%.11s' % creation_time)
        print(onlyDate)
    
# calculate number of images
No_images = len(list_images)
#print(No_images)
#print(list_images)
#print(images_location)

# Generate Sequence Numbers
NO =[]
for i in range(2000):
    if i < (len(list_images)):
        
        NO.append(i+1)
#print (NO)


# Create Dictionary
keys = ['NO', 'Name', 'Location']
subject = [NO,list_images,images_location]
report_dic = {z[0]:list(z[1:]) for z in zip(keys* len(subject),subject)}

#print (report_dic)
#print (len(report_dic))    
    

    
    
    

    
# Creating the PDF using ReportLab
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter, inch
from reportlab.platypus import Image, Paragraph, SimpleDocTemplate, Table
from reportlab.lib.styles import getSampleStyleSheet

doc = SimpleDocTemplate("Anomalies Detected.pdf", pagesize=letter)
#container for the 'Flowable' objects
elements = []
 

#Table 1
Ip = Image('pdo.jpg')
Ip.drawHeight = 1.25*inch*Ip.drawHeight / Ip.drawWidth
Ip.drawWidth = 1.25*inch

Ih = Image('haitch.png')
Ih.drawHeight = 1.25*inch*Ih.drawHeight / Ih.drawWidth
Ih.drawWidth = 1.25*inch


data1= [[Ip,'"Flowiness Anomalies Report"', Ih],[  ]]

 
t1=Table(data1,style=[('ALIGN',(0,0),(-1,-1),'CENTER'),
                    ('VALIGN',(0,0),(-1,-1),'MIDDLE'),
                    ('FONTSIZE', (0, 0), (-1, -1), 15),
                    ('BOX',(0,0),(-1,0),0.5,colors.black),
                    ('GRID',(0,0), (-1,0), 0.5,colors.black),
                    ])
t1._argW[2]=1.7*inch
t1._argW[1]=3.5*inch
t1._argW[0]=1.7*inch
 
elements.append(t1)


#space for Overview Map



#Table 2
data2= [["Date", onlyDate],
       ["Location", " ???"],
       ["Method of Inspection", "UAS"],
       ["Methos of Analysis", "AI"],
       ["Number of flights", " ??? flights"], #change Number of Flights
       ["Number of collected Images", (str(No_images)+" Images")],
       ["Flight Altitude Range", "50 - 100 m"],
       ["Report ID", "LEK???"]] # Change
 
t2=Table(data2,style=[('ALIGN',(0,0),(-1,-1),'LEFT'),
                    ('VALIGN',(0,0),(-1,-1),'MIDDLE'),
                    ('BOX',(0,0),(-1,-1),0.5,colors.black),
                    ('GRID',(0,0), (-1,-1), 0.5,colors.black),
                    
])
t2._argW[1]=3.5*inch
t2._argW[0]=3.5*inch

elements.append(t2)

style = getSampleStyleSheet()['Normal']
#style.leading = 24
#style.fontName = 'Helvetica'
#style.alignment = TA_JUSTIFY

style.spaceAfter = 24
space1 = Paragraph('''''',style)
elements.append(space1)


#Table 3
def table3(no, location):
    
    data3= [["No","Anomaly", "Location Coordinates", "Estimated Area", "Old/New leak"],
           [str(no), " Oil Spill",str(location)," ???"," ???"]]

 
    t3=Table(data3,style=[('ALIGN',(0,0),(-1,-1),'CENTER'),
                    ('VALIGN',(0,0),(-1,-1),'MIDDLE'),
                    ('BOX',(0,0),(-1,-1),0.5,colors.black),
                    ('GRID',(0,0), (-1,-1), 0.5,colors.black),
                    ])
    t3._argW[4]=1.5*inch
    t3._argW[3]=1.0*inch
    t3._argW[2]=3.0*inch
    t3._argW[1]=1.0*inch
    t3._argW[0]=0.5*inch
    
    return t3


table3_1= table3((report_dic["NO"][0][2]),(report_dic["Location"][0][2]))

elements.append(table3_1)
space2 = Paragraph('''''',style)
elements.append(space2)

table3_2= table3(report_dic["NO"][0][4],report_dic["Location"][0][4])
elements.append(table3_2)


# write the document to disk
doc.build(elements)


2019:11:03 
